In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from docx import Document
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
import json
import os
import gc


ModuleNotFoundError: No module named 'docx'

In [3]:
df = pd.read_csv("/home/macab/research/BloomNet/data/BloomTaxonomy.csv")

In [74]:
df['Label'].value_counts()

Comprehension    100
Knowledge        100
Application      100
Analysis         100
Synthesis        100
Evaluation       100
Name: Label, dtype: int64

In [76]:
df[20:25]

,Text,Label
20,How much water does an elephant drink,Knowledge
21,Identify a phrase as a simile,Knowledge
22,"Identify a property (commutative, additive, etc)",Knowledge
23,Identify fractions from a pictorial representa...,Knowledge
24,Identify ordinal positions,Knowledge


In [58]:
## create a label2target and target2label dictionary
encoder = LabelEncoder().fit(df['Label'])

In [59]:
encoder.classes_

array(['Analysis', 'Application', 'Comprehension', 'Evaluation',
       'Knowledge', 'Synthesis'], dtype=object)

In [60]:
label2idx  = dict(zip(encoder.classes_, list(range(0, 6, 1))))

In [65]:
idx2label = dict(zip(list(range(0, 6, 1)), encoder.classes_))

In [66]:
print(label2idx)

{'Analysis': 0, 'Application': 1, 'Comprehension': 2, 'Evaluation': 3, 'Knowledge': 4, 'Synthesis': 5}


In [67]:
print(idx2label)

{0: 'Analysis', 1: 'Application', 2: 'Comprehension', 3: 'Evaluation', 4: 'Knowledge', 5: 'Synthesis'}


In [68]:
with open("./label-dictionaries/label2idx.json", "w") as file:
    json.dump(label2idx, file)
with open("./label-dictionaries/idx2label.json", "w") as file:
    json.dump(idx2label, file)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
max(df['Text'].apply(lambda x:len(tokenizer.tokenize(x))))

44

In [9]:
df['Label'].value_counts()

Evaluation       100
Application      100
Analysis         100
Knowledge        100
Synthesis        100
Comprehension    100
Name: Label, dtype: int64

In [12]:
df = pd.read_csv("/home/macab/research/BloomNet/data/BloomTaxonomy2.csv")

In [15]:
sum(df['Label'].value_counts())

141

In [1]:
from dataset.dataset import BloomDataset
from transformers import AutoTokenizer
from config import config
import torch
from sklearn.model_selection import StratifiedKFold, KFold
import torch.nn as nn
from models.baselines.LSTM import LSTMClassifier
from dataset.loader import get_loaders

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
dataset = BloomDataset(
    tokenizer=tokenizer, 
    file_name="/home/macab/research/BloomNet/data/BloomTaxonomy.csv", 
    text_field=config['data']['Text'], 
    label_field=config['data']['Label'], 
    max_len=config['data']['max_len']
)

Using custom data configuration default-f2a03dde2a5e9193
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-f2a03dde2a5e9193/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [4]:
# dataset[0]

In [5]:
kfold = KFold(n_splits=4, shuffle=True, random_state=42)

In [6]:
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

      # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    

    print(f'fold = {fold} | trainlaoder = {len(trainloader)*10} | testloader = {len(testloader)*10}')

FOLD 0
--------------------------------
fold = 0 | trainlaoder = 450 | testloader = 150
FOLD 1
--------------------------------
fold = 1 | trainlaoder = 450 | testloader = 150
FOLD 2
--------------------------------
fold = 2 | trainlaoder = 450 | testloader = 150
FOLD 3
--------------------------------
fold = 3 | trainlaoder = 450 | testloader = 150


In [7]:
dataset2 = BloomDataset(
    tokenizer=tokenizer, 
    file_name="/home/macab/research/BloomNet/data/BloomTaxonomy2.csv", 
    text_field=config['data']['Text'], 
    label_field=config['data']['Label'], 
    max_len=config['data']['max_len']
)

Using custom data configuration default-9fbfd2cb2fcd0f38
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-9fbfd2cb2fcd0f38/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [8]:
batch = next(iter(trainloader))

In [9]:
batch['input_ids'].shape

torch.Size([10, 64])

In [10]:
model = LSTMClassifier(
    batch_size=10,
    output_size=6,
    hidden_size=768,
    vocab_size=tokenizer.vocab_size,
    embedding_length=768,
)

In [4]:
# logits = model.forward(input_sentence=batch['input_ids'])

In [1]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [3]:
# pack_padded_sequence?

In [2]:
import torch

In [1]:
# torch.nn.utils.rnn.pack_padded_sequence?

In [2]:
# torch.nn.utils.rnn.pack_padded_sequence

### BloomNet model

In [2]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification
from dataset.loader import get_loaders
from config import config
loaders = get_loaders(tokenizer=tokenizer, config=config['data'])
batch = next(iter(loaders['fold0']['train']))

In [3]:
tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

In [24]:
loaders = get_loaders(tokenizer=tokenizer, config=config['data'])
batch = next(iter(loaders['fold0']['train']))

Using custom data configuration default-0f07672984e5d615
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-0f07672984e5d615/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-873e9b92bfa129e8
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-873e9b92bfa129e8/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [25]:
batch = next(iter(loaders['fold0']['train']))

In [28]:
model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

In [29]:
outputs = model(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask']
)

In [39]:
outputs.shape

torch.Size([16, 768])

In [40]:
batch['text']

KeyError: 'text'

In [10]:
tokenizer.vocab_size

30522

In [11]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
bert_tokenizer.vocab_size

30522

In [13]:
if bert_tokenizer.vocab==tokenizer.vocab:
    print(True)
else:
    print(False)

True


In [14]:
tokenizer_ner = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

Downloading: 100%|██████████| 829/829 [00:00<00:00, 413kB/s]
Downloading: 100%|██████████| 213k/213k [00:03<00:00, 66.8kB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 2.61kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 104kB/s]
Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 53.0kB/s]


In [15]:
if bert_tokenizer.vocab==tokenizer_ner.vocab:
    print(True)
else:
    print(False)

False


In [16]:
tokenizer_ner.vocab_size

28996

In [19]:
tokenizer_ner.decode?

Signature:
tokenizer_ner.decode(
    token_ids: List[int],
    skip_special_tokens: bool = False,
    clean_up_tokenization_spaces: bool = True,
    spaces_between_special_tokens: bool = True,
) -> str
Docstring:
Converts a sequence of ids in a string, using the tokenizer and vocabulary
with options to remove special tokens and clean up tokenization spaces.

Similar to doing ``self.convert_tokens_to_string(self.convert_ids_to_tokens(token_ids))``.

Args:
    token_ids (:obj:`List[int]`):
        List of tokenized input ids. Can be obtained using the ``__call__`` method.
    skip_special_tokens (:obj:`bool`, `optional`, defaults to :obj:`False`):
        Whether or not to remove special tokens in the decoding.
    clean_up_tokenization_spaces (:obj:`bool`, `optional`, defaults to :obj:`True`):
        Whether or not to clean up the tokenization spaces.
    spaces_between_special_tokens (:obj:`bool`, `optional`, defaults to :obj:`True`):
        Whether or not to add spaces around specia

In [15]:
text =  batch['text']

In [16]:
# text

In [17]:
e = tokenizer.encode_plus(
    text=text,
    truncation=True,
    max_length=64,
    return_tensors="pt",
    return_attention_mask=True,
    padding="max_length",

)

In [42]:
tokenizer.decode(token_ids=batch['input_ids'][0].tolist(), skip_special_tokens=True)

'critique an experimental design or a research proposal.'

In [18]:
e['input_ids'].shape

torch.Size([1, 64])

In [22]:
import torch

In [25]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])

In [29]:
c = torch.cat((a, b), dim=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [28]:
c

tensor([1, 2, 3, 4, 5, 6])

Using custom data configuration default-0f07672984e5d615
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-0f07672984e5d615/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-873e9b92bfa129e8
Reusing dataset csv (/home/macab/.cache/huggingface/datasets/csv/default-873e9b92bfa129e8/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [14]:
# len(text)

In [6]:
pos = POSModel()

In [38]:
ner = NERModel()

Downloading: 100%|██████████| 433M/433M [03:14<00:00, 2.23MB/s]


In [7]:
pos_out = pos.forward(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask']
)

In [8]:
pos_out.shape

torch.Size([1, 768])

In [9]:
ner_out = pos.forward(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask']
)

In [10]:
print(ner_out.shape)

torch.Size([1, 768])


In [12]:
# batch['input_ids'].shape

In [1]:
import pytorch_lightning as pl

In [2]:
trainer = pl.Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [3]:
trainer.test?

Signature:
trainer.test(
    model: Union[pytorch_lightning.core.lightning.LightningModule, NoneType] = None,
    test_dataloaders: Union[torch.utils.data.dataloader.DataLoader, List[torch.utils.data.dataloader.DataLoader], NoneType] = None,
    ckpt_path: Union[str, NoneType] = 'best',
    verbose: bool = True,
    datamodule: Union[pytorch_lightning.core.datamodule.LightningDataModule, NoneType] = None,
)
Docstring:
Separates from fit to make sure you never run on your test set until you want to.

Args:
    ckpt_path: Either ``best`` or path to the checkpoint you wish to test.
        If ``None``, use the weights from the last epoch to test. Default to ``best``.

    datamodule: A instance of :class:`LightningDataModule`.

    model: The model to test.

    test_dataloaders: Either a single
        Pytorch Dataloader or a list of them, specifying validation samples.

    verbose: If True, prints the test results

Returns:
    The final test result dictionary. If no test_epoch_end is 

In [4]:
pl.__version__

'1.0.3'

In [5]:
pl.Trainer?

Init signature:
pl.Trainer(
    logger: Union[pytorch_lightning.loggers.base.LightningLoggerBase, Iterable[pytorch_lightning.loggers.base.LightningLoggerBase], bool] = True,
    checkpoint_callback: Union[pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint, bool] = True,
    callbacks: Union[List[pytorch_lightning.callbacks.base.Callback], NoneType] = None,
    default_root_dir: Union[str, NoneType] = None,
    gradient_clip_val: float = 0,
    process_position: int = 0,
    num_nodes: int = 1,
    num_processes: int = 1,
    gpus: Union[int, str, List[int], NoneType] = None,
    auto_select_gpus: bool = False,
    tpu_cores: Union[int, str, List[int], NoneType] = None,
    log_gpu_memory: Union[str, NoneType] = None,
    progress_bar_refresh_rate: int = 1,
    overfit_batches: Union[int, float] = 0.0,
    track_grad_norm: Union[int, float, str] = -1,
    check_val_every_n_epoch: int = 1,
    fast_dev_run: bool = False,
    accumulate_grad_batches: Union[int, Dict[int, int], L

In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [7]:
ckpt = ModelCheckpoint?

Init signature:
ModelCheckpoint(
    filepath: Union[str, NoneType] = None,
    monitor: Union[str, NoneType] = None,
    verbose: bool = False,
    save_last: Union[bool, NoneType] = None,
    save_top_k: Union[int, NoneType] = None,
    save_weights_only: bool = False,
    mode: str = 'auto',
    period: int = 1,
    prefix: str = '',
)
Docstring:     
Save the model after every epoch by monitoring a quantity.

After training finishes, use :attr:`best_model_path` to retrieve the path to the
best checkpoint file and :attr:`best_model_score` to retrieve its score.

Args:
    filepath: path to save the model file.
        Can contain named formatting options to be auto-filled.

        Example::

            # custom path
            # saves a file like: my/path/epoch=0.ckpt
            >>> checkpoint_callback = ModelCheckpoint('my/path/')

            # save any arbitrary metrics like `val_loss`, etc. in name
            # saves a file like: my/path/epoch=2-val_loss=0.02-other_metric=0

In [ ]:
ModelCheckpoint(
    filepath=os.path.join(ckpt_path),
    monitor=callback_config["monitor"],
    save_top_k=1,
    verbose=True,
    mode="mode"
)

In [9]:
pl.__version__

'1.0.3'

In [10]:
trainer.test?

Signature:
trainer.test(
    model: Union[pytorch_lightning.core.lightning.LightningModule, NoneType] = None,
    test_dataloaders: Union[torch.utils.data.dataloader.DataLoader, List[torch.utils.data.dataloader.DataLoader], NoneType] = None,
    ckpt_path: Union[str, NoneType] = 'best',
    verbose: bool = True,
    datamodule: Union[pytorch_lightning.core.datamodule.LightningDataModule, NoneType] = None,
)
Docstring:
Separates from fit to make sure you never run on your test set until you want to.

Args:
    ckpt_path: Either ``best`` or path to the checkpoint you wish to test.
        If ``None``, use the weights from the last epoch to test. Default to ``best``.

    datamodule: A instance of :class:`LightningDataModule`.

    model: The model to test.

    test_dataloaders: Either a single
        Pytorch Dataloader or a list of them, specifying validation samples.

    verbose: If True, prints the test results

Returns:
    The final test result dictionary. If no test_epoch_end is 